#  <center>LD computing</center>

This page documents the LD computing. <br>The input files: <br>&nbsp;&nbsp;&nbsp;&nbsp;1. vcf file with INFO and GT fields (multiallelic variant is not allowed) <br>&nbsp;&nbsp;&nbsp;&nbsp;2. sample list with integer ID for (CEU and YRI)</font><p>Computing steps: <br> &nbsp;&nbsp;&nbsp;&nbsp;1. Extract vcf file containing only good variants and samples in sample list file  <br> &nbsp;&nbsp;&nbsp;&nbsp;2. Set id field in vcf file as 'CHROM.POS.REF.ALT' <br> &nbsp;&nbsp;&nbsp;&nbsp;3. Generate Plink format files <br> &nbsp;&nbsp;&nbsp;&nbsp;4. LD computing <br> &nbsp;&nbsp;&nbsp;&nbsp;5. generate ld bed file <br>&nbsp;&nbsp;&nbsp;&nbsp; 6. sort and index LD bed file

## working directory and required file setup

In [4]:
import subprocess as sp

#working direcotry
workingDir = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/PLINK/LD2'

#vcf file directory
vcffileDir = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/bcf/SNV/matrix2VCF'
CHR = ['chr'+str(x+1) for x in range(22)]
vcffiles = {x:os.path.join(vcffileDir,x+'.vcf.gz') for x in CHR}

#sample list (integer ID)
#European sample ID
CEUID = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/PLINK/LD2/CEUID'
YRIID = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/PLINK/LD2/YRIID'
print('Setup Done')

#check if plink is in $PATH
ckrt = sp.run('plink',shell=True,stderr=sp.PIPE).stderr.decode('utf-8')
if 'command not found' in ckrt:
    print('Please load plink before you start jupyter notebook: <module load plink/1.90b>')


Setup Done
PLINK v1.90b3w 64-bit (3 Sep 2015)         https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3

  plink [input flag(s)...] {command flag(s)...} {other flag(s)...}
  plink --help {flag name(s)...}

Commands include --make-bed, --recode, --flip-scan, --merge-list,
--write-snplist, --list-duplicate-vars, --freqx, --missing, --test-mishap,
--hardy, --mendel, --ibc, --impute-sex, --indep-pairphase, --r2, --show-tags,
--blocks, --distance, --genome, --homozyg, --make-rel, --make-grm-gz,
--rel-cutoff, --cluster, --pca, --neighbour, --ibs-test, --regress-distance,
--model, --bd, --gxe, --logistic, --dosage, --lasso, --test-missing,
--make-perm-pheno, --tdt, --qfam, --annotate, --clump, --gene-report,
--meta-analysis, --epistasis, --fast-epistasis, and --score.

'plink --help | more' describes all functions (warning: long).


## 1. Extract vcf file containing only good variants and samples in sample list file

In [5]:
import os
import json

popsSample = {'ceu':CEUID,'yri':YRIID}
extVarFiles = []
for c in CHR:
    for p in popsSample:
        pDir = os.path.join(workingDir,p)
        if not os.path.isdir(pDir):
            os.system('mkdir -p ' + pDir)
        varFile = os.path.join(pDir,c+'.'+p+'.vcf.gz')
        extVarFiles.append(varFile)
        extCommand = 'bsub -q standard -R "rusage[mem=2000]"'
        extCommand += ' -oo ' + varFile + '.ext.log'
        extCommand += ' -eo ' + varFile + '.ext.elog'
        extCommand += ' bcftools view -S ' + popsSample[p]
        extCommand += ' -i ' + """QC='"Good"' -O z"""
        extCommand += ' -o ' + varFile
        extCommand += ' ' + vcffiles[c]
        os.system(extCommand)
    print('extracting good variants from ' + c)
#extracted variant file
with open('var.ext','w') as output:
    json.dump(extVarFiles,output)

Job <173912477> is submitted to queue <standard>.
extracting good variants from chr21
Job <173912478> is submitted to queue <standard>.
extracting good variants from chr22


## 2. Set id field in vcf file as 'CHROM.POS.REF.ALT'

In [7]:
import os
import json

with open('var.ext') as input:
    extVarFiles = json.load(input)
#check if vcf file containing only good variants and samples in sample list file available
for vf in extVarFiles:
    if not os.path.isfile(vf):
        print('vcf extracting above was not successful!')
        print('Please re-run')

#Set id field in vcf file as 'CHROM.POS.REF.ALT'
for vf in extVarFiles:
    idchgCommand = 'bsub -q standard -R "rusage[mem=2000]"'
    idchgCommand += ' -oo ' + vf + '.idchg.log'
    idchgCommand += ' -eo ' + vf + '.idchg.elog'
    idchgCommand += " bcftools annotate --set-id +\'%CHROM\.%POS\.%REF\.%ALT\' "+vf
    idchgCommand += ' -O z -o ' + vf + '.idchg.gz'
    os.system(idchgCommand)



Job <173916384> is submitted to queue <standard>.
Job <173916385> is submitted to queue <standard>.


## 3. Generate Plink format files

In [8]:
import os,sys
import json
import subprocess as sp

with open('var.ext') as input:
    extVarFiles = json.load(input)


#check if id field is replaced
for vf in extVarFiles:
    ckrt = sp.run('bcftools query -f "[%ID\n]" '+vf+ '.idchg.gz'+'|head -n 1',shell=True,stdout=sp.PIPE).stdout.decode('utf-8').strip()
    if ckrt == '.':
        print('ID replacement above was not successful!')
        print('please re-run')
        sys.exit(1)

#Generate Plink format files
for vf in extVarFiles:
    os.system('mv '+vf+ '.idchg.gz '+vf)
    vfDir,vfFile = os.path.split(vf)
    chrom = vfFile.split('.')[0]
    pfileCommand = 'bsub -q standard -R "rusage[mem=2000]"'
    pfileCommand += ' -oo ' + vf + '.pfile.log'
    pfileCommand += ' -eo ' + vf + '.pfile.elog'
    pfileCommand += ' plink --vcf ' + vf + ' --maf 0.05 --make-bed --out ' + os.path.join(vfDir,chrom)
    os.system(pfileCommand)

Job <173916934> is submitted to queue <standard>.
Job <173916935> is submitted to queue <standard>.


## 4. LD computing

In [10]:
import os,sys
import json

with open('var.ext') as input:
    extVarFiles = json.load(input)

#check if plink files generated
for vf in extVarFiles:
    vfDir,vfFile = os.path.split(vf)
    chrom = vfFile.split('.')[0]
    if not os.path.isfile(os.path.join(vfDir,chrom+'.fam')):
        print('plink files was not generated.')
        print('please re-run the step above')
        sys.exit(1)
#LD computing
for vf in extVarFiles:
    vfDir,vfFile = os.path.split(vf)
    chrom = vfFile.split('.')[0]
    ldCommand = 'bsub -q standard -R "rusage[mem=30000]"'
    ldCommand += ' -oo ' + vf + '.ld.log'
    ldCommand += ' -eo ' + vf + '.ld.elog'
    ldCommand += ' plink --bfile ' + os.path.join(vfDir,chrom)
    ldCommand += ' --r2 --ld-window-kb 200 --ld-window 99999 --ld-window-r2 0.1'
    ldCommand += ' --out ' + os.path.join(os.path.join(vfDir,chrom))
    os.system(ldCommand)
    
    

Job <173917334> is submitted to queue <standard>.
Job <173917336> is submitted to queue <standard>.


## 5. generate ld bed file

In [14]:
import os

if not os.path.isfile('genLdBed.py'):
    print('Please run scripts used for LD computing below')

for p in popsSample:
    pDir = os.path.join(workingDir,p)
    ldbedCommand = 'bsub -q standard -R "rusage[mem=30000]"'
    ldbedCommand += ' -oo ' + pDir + '/'+p+'.ldbed.log'
    ldbedCommand += ' -eo ' + pDir + '/'+p+'.ldbed.elog'
    ldbedCommand += ' python3 genLdBed.py '+pDir+' '+p
    os.system(ldbedCommand)

Job <173917988> is submitted to queue <standard>.


## 6. sort and index LD bed file

In [33]:
import os

if not os.path.isfile('sortbed.sh'):
    print('Please run scripts used for LD computing below')

for p in popsSample:
    ldbedFile = os.path.join(*[workingDir,p,p.upper()])
    sortindexCommand = 'bsub -q standard -R "rusage[mem=30000]"'
    sortindexCommand += ' -oo ' + pDir + '/'+p+'.sortindexldbed.log'
    sortindexCommand += ' -eo ' + pDir + '/'+p+'.sortindexldbed.elog'
    sortindexCommand += ' sh sortbed.sh ' + ldbedFile
    os.system(sortindexCommand)

Job <173920050> is submitted to queue <standard>.


## Scripts used for LD computing

In [21]:
%%writefile genLdBed.py

#!/usr/bin/python3

"""
Generate sorted and indexed ld file 
	input: direcotry where ld files for each chromosome generated from plink were located
	output: A single ld bed file
"""

import sys
import os
import re

ldDir = sys.argv[1]
pop = sys.argv[2]
CHR = ['chr'+str(x+1) for x in range(22)]
out = open(os.path.join(ldDir,pop.upper()),'w')


for c in CHR:
    ldfile = os.path.join(ldDir,c+'.ld')
    if not os.path.isfile(ldfile):
        print(ldfile + ' does not exist!')
        sys.exit(1)
    fh = open(ldfile)
    fh.readline()
    for line in fh:
        l = re.split('\s+',line.strip())
        chrom = 'chr'+l[0]
        p1 = str(int(l[1])-1)
        p2 = str(int(l[4])-1)
        out.write('\t'.join([chrom,l[1],l[4],l[2][-3:],l[5][-3:],l[-1]])+'\n')
    fh.close()

Overwriting genLdBed.py


In [32]:
%%writefile sortbed.sh

#!/bin/bash

file=$1


sort -k1,1 -k2,2n $file >${file}.sort
mv ${file}.sort $file
bgzip $file
tabix -p bed ${file}.gz

Overwriting sortbed.sh
